In [1]:
# import statements
from time import sleep
from json import dumps
from kafka import KafkaProducer
import random
from pprint import pprint
from datetime import datetime, timedelta

In [2]:
def read_hotspot_aqua_streaming_data():
    hotspot_aqua_streaming_lst=[]
    with open('hotspot_AQUA_streaming.csv') as read_file:
        read_file.readline() # for skipping header
        aqua=read_file.readlines()
        for a in aqua:
             # create dummy doc
            tmp_doc={
            "latitude":0.0,
            "date":"2021-12-27",
            "time":"04:16:51",
            "longitude":0.0,
            "confidence":0,
            "surface_temperature_celcius":0,
            "key":"aqua"
            }
            # update tmp_doc
            lst=a.split(',')
            #print(lst)
            tmp_doc["latitude"]=float(lst[0])
            tmp_doc["longitude"]=float(lst[1])
            tmp_doc["confidence"]=int(lst[2])
            tmp_doc["surface_temperature_celcius"]=int(lst[3].split('\n')[0])
            
            
            # update hotspot_aqua_streaming_lst
            hotspot_aqua_streaming_lst.append(tmp_doc)
    return hotspot_aqua_streaming_lst

In [3]:
print(len(read_hotspot_aqua_streaming_data()))

2644


In [4]:
def publish_message(producer_instance, topic_name, key, value):
    try:
        key_bytes = bytes(key, encoding='utf-8')
        value_bytes = bytes(value, encoding='utf-8')
        producer_instance.send(topic_name, key=key_bytes, value=value_bytes)
        producer_instance.flush()
        print('Message published successfully. Data: ' + str(value))
    except Exception as ex:
        print('Exception in publishing message.')
        print(str(ex))

In [5]:
def connect_kafka_producer():
    _producer = None
    try:
        _producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                                  api_version=(0, 10))
    except Exception as ex:
        print('Exception while connecting Kafka.')
        print(str(ex))
    finally:
        return _producer

In [6]:
if __name__ == '__main__':
   
    topic = 'Assgn'
    
    # retrieving data from csv file
    hotspot_aqua_streaming_lst=read_hotspot_aqua_streaming_data()
    
    
    # check size of hotspot lst
    size= len(hotspot_aqua_streaming_lst)
    
    print('Publishing records..')
    producer_aqua = connect_kafka_producer()
    
 
    # set random_index
    random_index=0
    
    #set datetime_val
    datetime_val=datetime.strptime(hotspot_aqua_streaming_lst[random_index]['date'],'%Y-%m-%d') 
    
    
    # create a random list of index ranging from 0 to 2643 to help us randomly access a different
    # aqua data at each iteration
    random_lst=random.sample(range(0, 2643), 2463)
    for e in range(2644):
        # get random index value
        
        # update hotspot_aqua_streaming_lst's datetime attribute 
        # at random_index pos (if every 10 secs is equivalent to
        # 24 hours then 2 secs is equivalent to 4.8 hrs)
        if random_index>0:    
            datetime_val=datetime_val + timedelta(hours=4.8)
        
        
        # assign time to 'time' attribute of hotspot_aqua_streaming_lst
        t_time=datetime.strptime(str(datetime_val), '%Y-%m-%d %H:%M:%S')
        val="{:02d}:{:02d}:{:02d}".format(t_time.hour,
                                                      t_time.minute, t_time.second)
        #print(val)
        
        # assign 'time' value to the datetime attr at random index of lst
        hotspot_aqua_streaming_lst[random_lst[random_index]]['time']=str(val)
        
        
        # assign the updated position record to data
        data=hotspot_aqua_streaming_lst[random_lst[random_index]]
        
        
        # publish message
        publish_message(producer_aqua, topic, 'aqua', dumps(data))
        
        # increment random index
        random_index+=1
        
        # sleep of 2 second to simulate real-time streaming data
        sleep(2)

Publishing records..
Message published successfully. Data: {"latitude": -37.591, "date": "2021-12-27", "time": "00:00:00", "longitude": 142.593, "confidence": 77, "surface_temperature_celcius": 51, "key": "aqua"}
Message published successfully. Data: {"latitude": -37.2912, "date": "2021-12-27", "time": "04:48:00", "longitude": 143.543, "confidence": 66, "surface_temperature_celcius": 43, "key": "aqua"}
Message published successfully. Data: {"latitude": -37.46, "date": "2021-12-27", "time": "09:36:00", "longitude": 148.102, "confidence": 88, "surface_temperature_celcius": 65, "key": "aqua"}
Message published successfully. Data: {"latitude": -34.2714, "date": "2021-12-27", "time": "14:24:00", "longitude": 141.6197, "confidence": 74, "surface_temperature_celcius": 48, "key": "aqua"}
Message published successfully. Data: {"latitude": -37.227, "date": "2021-12-27", "time": "19:12:00", "longitude": 141.146, "confidence": 54, "surface_temperature_celcius": 31, "key": "aqua"}
Message published

Message published successfully. Data: {"latitude": -37.9048, "date": "2021-12-27", "time": "14:24:00", "longitude": 145.7738, "confidence": 76, "surface_temperature_celcius": 49, "key": "aqua"}
Message published successfully. Data: {"latitude": -36.6963, "date": "2021-12-27", "time": "19:12:00", "longitude": 141.8645, "confidence": 79, "surface_temperature_celcius": 52, "key": "aqua"}
Message published successfully. Data: {"latitude": -37.8777, "date": "2021-12-27", "time": "00:00:00", "longitude": 143.371, "confidence": 81, "surface_temperature_celcius": 54, "key": "aqua"}
Message published successfully. Data: {"latitude": -37.6683, "date": "2021-12-27", "time": "04:48:00", "longitude": 143.3466, "confidence": 77, "surface_temperature_celcius": 51, "key": "aqua"}
Message published successfully. Data: {"latitude": -37.367, "date": "2021-12-27", "time": "09:36:00", "longitude": 148.04, "confidence": 71, "surface_temperature_celcius": 55, "key": "aqua"}
Message published successfully. Da

Message published successfully. Data: {"latitude": -34.351, "date": "2021-12-27", "time": "04:48:00", "longitude": 141.6567, "confidence": 99, "surface_temperature_celcius": 86, "key": "aqua"}
Message published successfully. Data: {"latitude": -36.7001, "date": "2021-12-27", "time": "09:36:00", "longitude": 141.7567, "confidence": 55, "surface_temperature_celcius": 39, "key": "aqua"}
Message published successfully. Data: {"latitude": -36.8283, "date": "2021-12-27", "time": "14:24:00", "longitude": 143.4605, "confidence": 100, "surface_temperature_celcius": 93, "key": "aqua"}
Message published successfully. Data: {"latitude": -37.38, "date": "2021-12-27", "time": "19:12:00", "longitude": 149.334, "confidence": 61, "surface_temperature_celcius": 31, "key": "aqua"}
Message published successfully. Data: {"latitude": -36.0942, "date": "2021-12-27", "time": "00:00:00", "longitude": 145.5705, "confidence": 61, "surface_temperature_celcius": 53, "key": "aqua"}
Message published successfully. D

Message published successfully. Data: {"latitude": -36.1376, "date": "2021-12-27", "time": "19:12:00", "longitude": 145.84, "confidence": 76, "surface_temperature_celcius": 55, "key": "aqua"}
Message published successfully. Data: {"latitude": -36.9203, "date": "2021-12-27", "time": "00:00:00", "longitude": 143.4581, "confidence": 58, "surface_temperature_celcius": 40, "key": "aqua"}
Message published successfully. Data: {"latitude": -35.9839, "date": "2021-12-27", "time": "04:48:00", "longitude": 143.6719, "confidence": 57, "surface_temperature_celcius": 47, "key": "aqua"}
Message published successfully. Data: {"latitude": -37.9538, "date": "2021-12-27", "time": "09:36:00", "longitude": 143.1487, "confidence": 62, "surface_temperature_celcius": 41, "key": "aqua"}
Message published successfully. Data: {"latitude": -35.89, "date": "2021-12-27", "time": "14:24:00", "longitude": 145.606, "confidence": 84, "surface_temperature_celcius": 59, "key": "aqua"}
Message published successfully. Dat

Message published successfully. Data: {"latitude": -36.1734, "date": "2021-12-27", "time": "09:36:00", "longitude": 141.3646, "confidence": 72, "surface_temperature_celcius": 47, "key": "aqua"}
Message published successfully. Data: {"latitude": -35.1383, "date": "2021-12-27", "time": "14:24:00", "longitude": 141.8172, "confidence": 50, "surface_temperature_celcius": 53, "key": "aqua"}
Message published successfully. Data: {"latitude": -36.0754, "date": "2021-12-27", "time": "19:12:00", "longitude": 141.6716, "confidence": 72, "surface_temperature_celcius": 46, "key": "aqua"}
Message published successfully. Data: {"latitude": -36.369, "date": "2021-12-27", "time": "00:00:00", "longitude": 143.7132, "confidence": 97, "surface_temperature_celcius": 80, "key": "aqua"}
Message published successfully. Data: {"latitude": -37.497, "date": "2021-12-27", "time": "04:48:00", "longitude": 147.266, "confidence": 70, "surface_temperature_celcius": 37, "key": "aqua"}
Message published successfully. D

...

Message published successfully. Data: {"latitude": -36.2646, "date": "2021-12-27", "time": "09:36:00", "longitude": 141.0021, "confidence": 82, "surface_temperature_celcius": 55, "key": "aqua"}
Message published successfully. Data: {"latitude": -36.7937, "date": "2021-12-27", "time": "14:24:00", "longitude": 141.3669, "confidence": 66, "surface_temperature_celcius": 43, "key": "aqua"}
Message published successfully. Data: {"latitude": -37.1839, "date": "2021-12-27", "time": "19:12:00", "longitude": 143.8112, "confidence": 89, "surface_temperature_celcius": 64, "key": "aqua"}
Message published successfully. Data: {"latitude": -36.7145, "date": "2021-12-27", "time": "00:00:00", "longitude": 143.471, "confidence": 78, "surface_temperature_celcius": 52, "key": "aqua"}
Message published successfully. Data: {"latitude": -37.5402, "date": "2021-12-27", "time": "04:48:00", "longitude": 143.2559, "confidence": 76, "surface_temperature_celcius": 50, "key": "aqua"}
Message published successfully.

Message published successfully. Data: {"latitude": -34.2695, "date": "2021-12-27", "time": "00:00:00", "longitude": 142.2025, "confidence": 80, "surface_temperature_celcius": 54, "key": "aqua"}
Message published successfully. Data: {"latitude": -36.2779, "date": "2021-12-27", "time": "04:48:00", "longitude": 145.7794, "confidence": 72, "surface_temperature_celcius": 47, "key": "aqua"}
Message published successfully. Data: {"latitude": -37.1378, "date": "2021-12-27", "time": "09:36:00", "longitude": 141.9545, "confidence": 81, "surface_temperature_celcius": 54, "key": "aqua"}
Message published successfully. Data: {"latitude": -36.6962, "date": "2021-12-27", "time": "14:24:00", "longitude": 145.2642, "confidence": 84, "surface_temperature_celcius": 58, "key": "aqua"}
Message published successfully. Data: {"latitude": -36.3066, "date": "2021-12-27", "time": "19:12:00", "longitude": 141.0728, "confidence": 81, "surface_temperature_celcius": 54, "key": "aqua"}
Message published successfully

Message published successfully. Data: {"latitude": -37.446, "date": "2021-12-27", "time": "14:24:00", "longitude": 142.8978, "confidence": 72, "surface_temperature_celcius": 47, "key": "aqua"}
Message published successfully. Data: {"latitude": -36.778, "date": "2021-12-27", "time": "19:12:00", "longitude": 145.1635, "confidence": 69, "surface_temperature_celcius": 45, "key": "aqua"}
Message published successfully. Data: {"latitude": -36.4547, "date": "2021-12-27", "time": "00:00:00", "longitude": 143.145, "confidence": 74, "surface_temperature_celcius": 48, "key": "aqua"}
Message published successfully. Data: {"latitude": -37.3911, "date": "2021-12-27", "time": "04:48:00", "longitude": 147.037, "confidence": 61, "surface_temperature_celcius": 50, "key": "aqua"}
Message published successfully. Data: {"latitude": -36.4315, "date": "2021-12-27", "time": "09:36:00", "longitude": 141.1987, "confidence": 76, "surface_temperature_celcius": 49, "key": "aqua"}
Message published successfully. Da

Message published successfully. Data: {"latitude": -37.1009, "date": "2021-12-27", "time": "04:48:00", "longitude": 141.9666, "confidence": 69, "surface_temperature_celcius": 45, "key": "aqua"}
Message published successfully. Data: {"latitude": -38.1665, "date": "2021-12-27", "time": "09:36:00", "longitude": 143.062, "confidence": 100, "surface_temperature_celcius": 124, "key": "aqua"}
Message published successfully. Data: {"latitude": -36.9632, "date": "2021-12-27", "time": "14:24:00", "longitude": 141.3474, "confidence": 81, "surface_temperature_celcius": 54, "key": "aqua"}
Message published successfully. Data: {"latitude": -36.8298, "date": "2021-12-27", "time": "19:12:00", "longitude": 143.4483, "confidence": 96, "surface_temperature_celcius": 79, "key": "aqua"}
Message published successfully. Data: {"latitude": -36.998, "date": "2021-12-27", "time": "00:00:00", "longitude": 148.1994, "confidence": 73, "surface_temperature_celcius": 48, "key": "aqua"}
Message published successfully

KeyboardInterrupt: 